<a href="https://colab.research.google.com/github/prafulr89/ML_to_predict_SDN_attacks/blob/main/sdn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>